In [ ]:
import pandas as pd
import numpy as np
import heapq
import matplotlib.pyplot as plt
import sklearn.model_selection as ml


In [ ]:
cancer_dataset = pd.read_csv("datasets/cancer.csv")
cancer_labels = [1 if val == 'M' else 0 for val in cancer_dataset['label'].values]
cancer_points = cancer_dataset.drop(['label'], axis=1).values


In [ ]:
spam_dataset = pd.read_csv("datasets/spam.csv")
spam_labels = [val for val in spam_dataset['label'].values]
spam_points = spam_dataset.drop(['label'], axis=1).values
